In [9]:
# Import Required Libraries
import pandas as pd
import numpy as np
import warnings
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegressionCV
from imblearn.over_sampling import SMOTE
import os

warnings.filterwarnings('ignore')

# Set up paths
ROOT = os.path.dirname(os.path.abspath('scaled_eda_data.csv'))
DATA_PATH = 'scaled_eda_data.csv'
TARGET = 'is_late'
print(' Using scaled EDA dataset for feature engineering')

 Using scaled EDA dataset for feature engineering


In [10]:
# Load the scaled data from EDA
df = pd.read_csv(DATA_PATH)
print('Dataset shape:', df.shape)
print('Source: scaled_eda_data.csv (from EDA Analysis)')
print('\nColumn data types:')
print(df.dtypes.value_counts())
print('\n' + '='*60)
print('MISSING VALUES (Top 20 columns):')
print('='*60)
missing_counts = df.isnull().sum().sort_values(ascending=False)
if missing_counts.sum() > 0:
    print(missing_counts.head(20))
    print(f'\nTotal columns with missing values: {missing_counts[missing_counts > 0].shape[0]}')
else:
    print(' No missing values found (already handled in EDA)')

# Analyze target variable
print('\n' + '='*60)
print('TARGET VARIABLE DISTRIBUTION:')
print('='*60)
print(df[TARGET].value_counts(dropna=False))
print(f'\nTotal records: {len(df)}')

# Calculate class imbalance ratio
value_counts = df[TARGET].value_counts()
max_class = value_counts.max()
min_class = value_counts.min()
imbalance_ratio = max_class / min_class
print(f'\nClass Imbalance Ratio: {imbalance_ratio:.2f}:1')

if imbalance_ratio > 1.5:
    apply_smote = True
    print('  Class imbalance detected! SMOTE will be applied.')
else:
    apply_smote = False
    print(' Data is relatively balanced. SMOTE may not be necessary.')

Dataset shape: (2832, 19)
Source: scaled_eda_data.csv (from EDA Analysis)

Column data types:
float64    10
object      7
bool        2
Name: count, dtype: int64

MISSING VALUES (Top 20 columns):
restaurant_avg_prep_min    141
delivery_partner_orders    133
order_value_inr            125
distance_km                123
delivery_partner_rating     93
restaurant_rating           80
id                           0
day_of_week                  0
order_hour                   0
cuisine_type                 0
restaurant_type              0
weather_condition            0
is_peak_hour                 0
num_items                    0
estimated_delivery_min       0
traffic_density              0
is_promo_order               0
area_type                    0
is_late                      0
dtype: int64

Total columns with missing values: 6

TARGET VARIABLE DISTRIBUTION:
is_late
on_time    1717
late       1115
Name: count, dtype: int64

Total records: 2832

Class Imbalance Ratio: 1.54:1
  Class imbalan

In [11]:
# Missing value check (should be minimal from EDA)
print('='*60)
print('MISSING VALUE CHECK:')
print('='*60)

total_records = len(df)
missing_total = df.isnull().sum().sum()
print(f'Total records: {total_records}')
print(f'Total missing values in dataset: {missing_total}')

if missing_total == 0:
    print('\n No missing values - dataset already cleaned by EDA pipeline')
    df_clean = df.copy()
    strategy_used = 'Pre-cleaned by EDA'
else:
    # Strategy 1: Remove all rows with any missing values
    df_no_missing = df.dropna()
    retention_pct = (len(df_no_missing) / total_records) * 100
    
    print(f'\nRecords after removing ALL missing values: {len(df_no_missing)}')
    print(f'Data retention: {retention_pct:.2f}%')
    
    # Decide strategy
    if retention_pct >= 95:
        print('\n Strategy: Remove all rows with ANY missing values')
        print(f'  Reason: {retention_pct:.2f}% data retention (>= 95%)')
        df_clean = df.dropna()
        rows_dropped = total_records - len(df_clean)
        print(f'  Rows dropped: {rows_dropped}')
        strategy_used = 'Drop all missing'
    else:
        print('\n Strategy: Remove rows with 4 or more missing fields')
        print(f'  Reason: {retention_pct:.2f}% data retention (< 95%)')
        # Count missing values per row
        missing_per_row = df.isnull().sum(axis=1)
        df_clean = df[missing_per_row < 4]
        rows_dropped = total_records - len(df_clean)
        print(f'  Rows dropped: {rows_dropped}')
        strategy_used = 'Drop rows with 4+ missing'

print(f'\nRecords after cleaning: {len(df_clean)}')
print(f'Data retention: {(len(df_clean) / total_records) * 100:.2f}%')

MISSING VALUE CHECK:
Total records: 2832
Total missing values in dataset: 695

Records after removing ALL missing values: 2185
Data retention: 77.15%

 Strategy: Remove rows with 4 or more missing fields
  Reason: 77.15% data retention (< 95%)
  Rows dropped: 0

Records after cleaning: 2832
Data retention: 100.00%


In [12]:
# Prepare data for SMOTE and feature engineering
y = df_clean[TARGET].copy()
X = df_clean.drop(columns=[TARGET])

# Encode target variable if necessary
if y.dtype == object or y.dtype.name == 'category':
    le_target = LabelEncoder()
    y = pd.Series(le_target.fit_transform(y.astype(str)), index=y.index)

print('\n' + '='*60)
print('CLASS DISTRIBUTION BEFORE SMOTE:')
print('='*60)
print(y.value_counts())

# Apply SMOTE if needed
if apply_smote:
    print('\nApplying SMOTE to balance classes...')
    # Identify numeric and categorical columns
    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()
    
    # Impute before SMOTE (SMOTE works on numeric data)
    if numeric_cols:
        num_imputer = SimpleImputer(strategy='median')
        X_num = pd.DataFrame(num_imputer.fit_transform(X[numeric_cols]), columns=numeric_cols)
    
    if cat_cols:
        X_cat = X[cat_cols].fillna('missing')
        X_cat = pd.get_dummies(X_cat, drop_first=True)
    
    X_processed = pd.concat([X_num, X_cat], axis=1) if cat_cols else X_num
    X_processed = X_processed.fillna(0)
    
    # Apply SMOTE
    smote = SMOTE(random_state=42, k_neighbors=5)
    try:
        X_smoted, y_smoted = smote.fit_resample(X_processed, y)
        X_smoted = pd.DataFrame(X_smoted, columns=X_processed.columns)
        y_smoted = pd.Series(y_smoted, index=range(len(y_smoted)))
        print(' SMOTE applied successfully!')
        print('\nCLASS DISTRIBUTION AFTER SMOTE:')
        print(y_smoted.value_counts())
        
        # Update for next sections
        X = X_smoted
        y = y_smoted
        df_for_modeling = X.copy()
        df_for_modeling.insert(0, TARGET, y.values)
    except Exception as e:
        print(f'⚠️  SMOTE application failed: {e}')
        print('Proceeding without SMOTE')
        df_for_modeling = df_clean.copy()
else:
    print('\nSMOTE not applied (no significant class imbalance detected)')


CLASS DISTRIBUTION BEFORE SMOTE:
1    1717
0    1115
Name: count, dtype: int64

Applying SMOTE to balance classes...
 SMOTE applied successfully!

CLASS DISTRIBUTION AFTER SMOTE:
1    1717
0    1717
Name: count, dtype: int64


In [13]:
# Prepare features for Ridge Regression
print('='*60)
print('FEATURE IMPORTANCE WITH RIDGE REGRESSION:')
print('='*60)

if not apply_smote:
    # If SMOTE was not applied, prepare data now
    y = df_clean[TARGET].copy()
    X = df_clean.drop(columns=[TARGET])
    
    if y.dtype == object or y.dtype.name == 'category':
        le_target = LabelEncoder()
        y = pd.Series(le_target.fit_transform(y.astype(str)), index=y.index)

# Identify numeric and categorical columns
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

print(f'\nNumeric columns: {len(numeric_cols)}')
print(f'Categorical columns: {len(cat_cols)}')

# Handle numeric columns - impute with median
X_numeric = X[numeric_cols].copy() if numeric_cols else pd.DataFrame()
if numeric_cols:
    num_imputer = SimpleImputer(strategy='median')
    X_numeric = pd.DataFrame(num_imputer.fit_transform(X_numeric), columns=numeric_cols)

# Handle categorical columns - impute and one-hot encode
X_categorical = pd.DataFrame() if not cat_cols else X[cat_cols].fillna('missing')
if cat_cols:
    X_categorical = pd.get_dummies(X_categorical.astype(str), drop_first=True)

# Combine processed features
if X_categorical.shape[1] > 0:
    X_processed = pd.concat([X_numeric, X_categorical], axis=1)
else:
    X_processed = X_numeric

# Fill any remaining NaNs
X_processed = X_processed.fillna(0)
print(f'\nProcessed feature count: {X_processed.shape[1]}')

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_processed)

# Fit Ridge Regression (L2 penalty) for feature importance
print('\nFitting LogisticRegression with Ridge (L2) penalty...')
try:
    ridge_model = LogisticRegressionCV(cv=5, penalty='l2', solver='lbfgs', 
                                       scoring='roc_auc', max_iter=5000, n_jobs=-1)
    ridge_model.fit(X_scaled, y)
    ridge_coef = ridge_model.coef_.ravel()
    print(' Ridge model fitted successfully!')
    
    # Create coefficient dataframe
    coef_df = pd.DataFrame({
        'Feature': X_processed.columns,
        'Coefficient': ridge_coef,
        'Abs_Coefficient': np.abs(ridge_coef)
    })
    coef_df = coef_df.sort_values('Abs_Coefficient', ascending=False)
    
    print('\nTop 20 Features by Ridge Coefficient:')
    print(coef_df.head(20).to_string(index=False))
    
except Exception as e:
    print(f'⚠️  Ridge model fitting failed: {e}')
    coef_df = None

FEATURE IMPORTANCE WITH RIDGE REGRESSION:

Numeric columns: 10
Categorical columns: 32

Processed feature count: 42

Fitting LogisticRegression with Ridge (L2) penalty...
 Ridge model fitted successfully!

Top 20 Features by Ridge Coefficient:
                           Feature  Coefficient  Abs_Coefficient
    cuisine_type_pizza_burger_True    -0.444030         0.444030
          cuisine_type_indian_True    -0.413575         0.413575
         cuisine_type_dessert_True    -0.397745         0.397745
    restaurant_type_fast_food_True    -0.395602         0.395602
    cuisine_type_south_indian_True    -0.374158         0.374158
         cuisine_type_chinese_True    -0.373291         0.373291
restaurant_type_casual_dining_True    -0.364227         0.364227
restaurant_type_cloud_kitchen_True    -0.330676         0.330676
           cuisine_type_other_True    -0.320017         0.320017
         cuisine_type_italian_True    -0.309445         0.309445
  restaurant_type_fine_dining_True    -0.

In [14]:
# Drop low importance features
print('='*60)
print('LOW IMPORTANCE FEATURE REMOVAL:')
print('='*60)

if coef_df is not None:
    # Calculate 10th percentile threshold
    threshold = np.percentile(coef_df['Abs_Coefficient'], 10)
    print(f'\n10th Percentile Threshold: {threshold:.6f}')
    
    # Identify features to drop
    features_to_drop = coef_df[coef_df['Abs_Coefficient'] <= threshold]['Feature'].tolist()
    print(f'\nTotal features to drop: {len(features_to_drop)}')
    print(f'Features to keep: {len(coef_df) - len(features_to_drop)}')
    
    print('\n' + '='*60)
    print('FEATURES BEING DROPPED:')
    print('='*60)
    drop_info = coef_df[coef_df['Abs_Coefficient'] <= threshold][['Feature', 'Abs_Coefficient']]
    print(drop_info.to_string(index=False))
    
    # Drop features from processed data
    X_final = X_processed.drop(columns=features_to_drop)
    print(f'\n Features removed. Final feature count: {X_final.shape[1]}')
    
else:
    print('  Ridge model not available. Using all features.')
    features_to_drop = []
    X_final = X_processed.copy()

LOW IMPORTANCE FEATURE REMOVAL:

10th Percentile Threshold: 0.017687

Total features to drop: 5
Features to keep: 37

FEATURES BEING DROPPED:
                Feature  Abs_Coefficient
 estimated_delivery_min         0.016190
delivery_partner_rating         0.014260
                     id         0.014247
              num_items         0.013586
        order_value_inr         0.009220

 Features removed. Final feature count: 37


In [15]:
# Save the engineered dataset to Excel
print('='*60)
print('SAVING ENGINEERED DATASET:')
print('='*60)

# Create final dataframe with target variable
df_final = X_final.copy()
df_final.insert(0, TARGET, y.values)

# Define output file path
output_file = 'engineered_features.xlsx'

try:
    # Save to Excel
    df_final.to_excel(output_file, index=False, sheet_name='Features')
    print(f'\n Dataset saved successfully!')
    print(f'File: {output_file}')
    print(f'Shape: {df_final.shape}')
    print(f'Rows: {df_final.shape[0]}, Columns: {df_final.shape[1]}')
    
    # Summary statistics
    print('\n' + '='*60)
    print('SUMMARY STATISTICS:')
    print('='*60)
    print(f'Source dataset: scaled_eda_data.csv')
    print(f'After missing value check: {df_clean.shape}')
    if apply_smote:
        print(f'After SMOTE: {df_final.shape[0]} rows')
    print(f'Final engineered dataset: {df_final.shape}')
    print(f'Features removed: {len(features_to_drop)}')
    print(f'Features retained: {df_final.shape[1] - 1}')
    
    if coef_df is not None:
        # Save coefficient information to separate sheet
        with pd.ExcelWriter(output_file, mode='a', engine='openpyxl') as writer:
            coef_df.to_excel(writer, sheet_name='Feature_Importance', index=False)
        print(f'\n Feature importance scores saved to "Feature_Importance" sheet')
    
except Exception as e:
    print(f'  Error saving file: {e}')

print('\n Feature Engineering Pipeline Complete!')
print('Pipeline: scaled_eda_data.csv → SMOTE → Ridge Selection → engineered_features.xlsx')

SAVING ENGINEERED DATASET:

 Dataset saved successfully!
File: engineered_features.xlsx
Shape: (3434, 38)
Rows: 3434, Columns: 38

SUMMARY STATISTICS:
Source dataset: scaled_eda_data.csv
After missing value check: (2832, 19)
After SMOTE: 3434 rows
Final engineered dataset: (3434, 38)
Features removed: 5
Features retained: 37

 Feature importance scores saved to "Feature_Importance" sheet

 Feature Engineering Pipeline Complete!
Pipeline: scaled_eda_data.csv → SMOTE → Ridge Selection → engineered_features.xlsx
